In [1]:
import bs4 as bs
import datetime as dt
import os
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader import data as web
import pickle
import requests
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
style.use('ggplot')

In [2]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text[:-1]
    tickers.append(ticker)

In [10]:
start = dt.datetime(2015, 9, 7)
end = dt.datetime.now()
for ticker in sorted(tickers):
    df = web.DataReader(ticker, 'fred', start, end)
    df.to_csv('stock_dfs/{}.csv'.format(ticker))

RemoteDataError: Unable to read URL: https://fred.stlouisfed.org/graph/fredgraph.csv?id=A

In [15]:
def compile_data():

    main_df = pd.DataFrame()

    for count,ticker in enumerate(tickers):
        df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
        df.set_index('date', inplace=True)

        df.rename(columns={'close':ticker}, inplace=True)
        df.drop(['open','high','low','volume'],1,inplace=True)

        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how='outer')

        if count % 10 == 0:
            print(count)
    print(main_df.head())
    main_df.to_csv('sp500_joined_closes.csv')
    
compile_data()

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
                 MMM      ABT     ABBV   ABMD      ACN     ATVI   ADBE   AMD  \
date                                                                           
2014-09-08  131.0537  39.1953  48.2472  25.25  75.1709  23.1656  73.39  4.13   
2014-09-09  130.6648  39.2136  49.4453  24.94  74.4535  23.0394  72.42  4.08   
2014-09-10  130.8276  39.2777  49.6537  24.83  74.6926  23.1171  72.33  4.08   
2014-09-11  130.5653  38.9664  50.2267  25.28  74.9685  23.1753  72.42  4.08   
2014-09-12  130.1944  38.9572  50.2528  25.13  74.4719  22.7093  70.98  3.96   

                 AAP      AES   ...         WYNN      XEL      XRX     XLNX  \
date                            ...                                           
2014-09-08  135.8541  12.8438   ...     168.6007  28.0413  31.3292  38.8397   
2014-09-0